In [1]:
import numpy as np
import pandas as pd
import math
import requests 
import matplotlib as plt
import seaborn as sns
import yfinance as yf
import pandas_datareader as web
from pandas_datareader import data
from bs4 import BeautifulSoup as bs
from scipy import stats

In [2]:
tick = pd.read_csv("all_stocks")

In [16]:
final = pd.DataFrame(columns=['Ticker','Price','Trailing PE ratio', 'Forward PE ratio','Shares to buy'])
for i in range(len(tick["0"])):
    print("Producing data for: ", tick["0"][i])
    try:
        ticker = yf.Ticker(tick["0"][i])
        info = ticker.info
        for i in info: 
            if "price" in i.lower(): 
                print(i)
        #We have price to book
    except TypeError: 
        print("Nonetype found for: " + tick["0"][i])
        continue
    except IndexError: 
        print("Couldn't find: ",tick["0"][i])
        continue
    except KeyError:
        print("Couldnt find key for: " + tick['0'][i])
        continue
    break

    '''    
    try: 
        pass
    except IndexError: 
        continue
    #Change to keep only last 2 days!!! get date
    
    #Append latest value
    final = final.append(
        pd.Series(d,
        index=['Ticker','Price','Trailing PE ratio', 'Forward PE ratio','Shares to buy']), 
        ignore_index = True
    ) 
    '''

Producing data for:  AOS
targetLowPrice
targetMedianPrice
currentPrice
targetMeanPrice
targetHighPrice
priceToBook
priceHint
priceToSalesTrailing12Months
navPrice
strikePrice
regularMarketPrice
preMarketPrice


In [5]:
final.to_csv("value_invest_advanced_sheet")